<a href="https://colab.research.google.com/github/ahimsaj/ML-Training/blob/main/FakeNewsClassifierUsingBidirectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using Bidirectional LSTM

In [2]:
import pandas as pd

In [7]:
df=pd.read_csv('/content/train.csv')

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
###Drop Nan Values
df=df.dropna()


In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [12]:
## Get the Dependent features
y=df['label']

In [13]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [14]:
X.shape

(18285, 4)

In [15]:
y.shape

(18285,)

In [16]:
import tensorflow as tf

In [17]:
tf.__version__

'2.8.2'

In [18]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [19]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [20]:
messages=X.copy()

In [21]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [22]:
messages.reset_index(inplace=True)

In [23]:
import nltk
import re
from nltk.corpus import stopwords

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [26]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [27]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1009, 3825, 1743, 3444, 1966, 1764, 2084, 4086, 4570, 1949],
 [4482, 2158, 2472, 327, 799, 2761, 598],
 [1887, 4232, 4055, 1890],
 [4866, 2490, 2933, 4946, 3812, 996],
 [2685, 799, 1865, 306, 2124, 3936, 799, 184, 4135, 1158],
 [554,
  2193,
  1507,
  635,
  2343,
  3479,
  1364,
  4053,
  3995,
  2,
  1339,
  3506,
  3080,
  2247,
  598],
 [1993, 495, 3925, 1145, 326, 877, 2321, 1613, 1279, 512, 4491],
 [4598, 631, 3907, 4533, 1212, 439, 3479, 1696, 1279, 512, 4491],
 [816, 2732, 3220, 1985, 1078, 2656, 122, 2338, 3479, 580],
 [2713, 4536, 186, 2178, 467, 1447, 3392, 131],
 [277, 3972, 580, 410, 3500, 3648, 3898, 1134, 4729, 2496, 55],
 [4946, 3129, 1966, 2656, 3479, 1212],
 [2618, 3289, 3860, 4164, 803, 4762, 4113, 3916, 609],
 [1820, 448, 2710, 976, 4881, 1276, 4528, 1279, 512, 4491],
 [2827, 1768, 4198, 4646, 3843, 1279, 512, 4491],
 [820, 1430, 1208, 2572, 4893, 3932, 3466, 212, 211, 3934],
 [1634, 1670, 2158],
 [957, 1307, 182, 4806, 3479, 3146, 3751, 598],
 [4525, 2358, 2472, 

### Embedding Representation

In [36]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4086 4570 1949]
 [   0    0    0 ...  799 2761  598]
 [   0    0    0 ... 4232 4055 1890]
 ...
 [   0    0    0 ... 1279  512 4491]
 [   0    0    0 ... 3148 3205  462]
 [   0    0    0 ... 1248 3207 4064]]


In [37]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1009,
       3825, 1743, 3444, 1966, 1764, 2084, 4086, 4570, 1949], dtype=int32)

In [38]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
len(embedded_docs),y.shape

(18285, (18285,))

In [41]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [42]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [44]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 16s 58ms/step - loss: 0.2990 - accuracy: 0.8572 - val_loss: 0.2027 - val_accuracy: 0.9183
Epoch 2/10
192/192 [==============================] - 10s 52ms/step - loss: 0.1362 - accuracy: 0.9473 - val_loss: 0.2005 - val_accuracy: 0.9200
Epoch 3/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0962 - accuracy: 0.9660 - val_loss: 0.2819 - val_accuracy: 0.9142
Epoch 4/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0733 - accuracy: 0.9732 - val_loss: 0.2699 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0533 - accuracy: 0.9809 - val_loss: 0.3584 - val_accuracy: 0.9092
Epoch 6/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0363 - accuracy: 0.9873 - val_loss: 0.3827 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0247 - accuracy: 0.9922 - val_loss: 0.4201 - val_accuracy:

### Performance Metrics And Accuracy

In [50]:
predictions = (model.predict(X_test) > 0.5).astype("int32")

In [51]:
from sklearn.metrics import confusion_matrix

In [53]:
confusion_matrix(y_test,predictions)

array([[2883,  536],
       [1857,  759]])

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.6034797017398509

In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.61      0.84      0.71      3419
           1       0.59      0.29      0.39      2616

    accuracy                           0.60      6035
   macro avg       0.60      0.57      0.55      6035
weighted avg       0.60      0.60      0.57      6035

